# ECON7890 Assignment 4 ? Kowloon Property Transactions

End-to-end notebook: import/clean the 28hse Kowloon transactions (Nov 2014?Nov 2020), merge macro data, explore, model price drivers, and summarize findings.

**Tasks**
- Import & clean the raw CSV, fix headers, parse numbers, handle missing values
- Merge with external macro indicators (World Bank unemployment & CPI)
- Exploratory data analysis with descriptive stats and visuals
- Train/evaluate multiple models to explain price per saleable sqft
- Summarize insights and next steps


In [1]:
import re
from pathlib import Path
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

DATA_PATH = Path('Excercises/hw.csv')
OUTPUT_DIR = Path('Excercises/assignment4_outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'Excercises\\assignment4_outputs'

## 1) Load raw data
Read the provided `hw.csv` file and inspect the columns.


In [ ]:
raw_df = pd.read_csv(DATA_PATH)
display(raw_df.head())
print(f"Rows: {len(raw_df):,}, Columns: {len(raw_df.columns)}")
raw_df.columns.tolist()

## 2) Cleaning helpers
The raw headers are cryptic and some fields embed units/HTML. Helper functions below parse numbers, percentages, and holding periods.


In [ ]:
def _parse_number(text: Any) -> float:
    # Extract first numeric value from a string; return NaN on failure.
    if pd.isna(text):
        return np.nan
    match = re.search(r"([0-9]*\.?[0-9]+)", str(text))
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return np.nan
    return np.nan


def _parse_percent(text: Any) -> float:
    # Convert percent strings (e.g., '413%') to decimal (4.13).
    if pd.isna(text):
        return np.nan
    text = str(text).replace('%', '').strip()
    if not text or text == '--':
        return np.nan
    try:
        return float(text) / 100.0
    except ValueError:
        return np.nan


def _parse_holding_years(text: Any) -> float:
    # Convert holding period text such as '16 years 251 days' into years.
    if pd.isna(text):
        return np.nan
    if text in ('--', '-1'):
        return np.nan
    text = str(text)
    years = _parse_number(text)
    days_match = re.search(r"([0-9]+)\s*day", text)
    days = float(days_match.group(1)) if days_match else 0.0
    if np.isnan(years):
        return days / 365.0 if days else np.nan
    return years + days / 365.0

## 3) Clean transactions
- Drop unnamed columns
- Rename to readable headers
- Parse dates, numbers, areas, floor, holding period, win/loss
- Remove bad rows and duplicates


In [ ]:
def clean_transactions(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df = df.drop(columns=[c for c in df.columns if c.startswith('Unnamed')], errors='ignore')
    rename_map: Dict[str, str] = {
        'withpre': 'pre_sale_flag',
        'catname': 'estate',
        'catfathername': 'district',
        'price': 'price_label',
        'price_value': 'price_hkd',
        'holddate': 'holding_period_text',
        'winloss': 'winloss_pct',
        'act_area': 'saleable_area',
        'area': 'gross_area',
        'arearaw': 'gross_area_raw',
        'sq_price': 'sq_price_label',
        'sq_price_value': 'price_per_gross_sf',
        'sq_actprice': 'sq_actprice_label',
        'sq_actprice_value': 'price_per_saleable_sf',
        'date_y': 'year_text',
        'state': 'building',
        'addr': 'address',
    }
    df = df.rename(columns=rename_map)

    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['pre_sale_flag'] = df['pre_sale_flag'].fillna(0).astype(int)
    df['price_hkd'] = pd.to_numeric(df['price_hkd'], errors='coerce')
    df['price_per_gross_sf'] = pd.to_numeric(df['price_per_gross_sf'], errors='coerce')
    df['price_per_saleable_sf'] = pd.to_numeric(df['price_per_saleable_sf'], errors='coerce')

    df['saleable_area'] = df['saleable_area'].apply(_parse_number)
    df['gross_area'] = df['gross_area'].apply(_parse_number)
    gross_series = df['gross_area_raw'] if 'gross_area_raw' in df.columns else pd.Series(np.nan, index=df.index)
    df['gross_area_raw'] = pd.to_numeric(gross_series, errors='coerce')

    df['floor_num'] = df['floor'].apply(_parse_number)
    df['holding_period_years'] = df['holding_period_text'].apply(_parse_holding_years)
    df['winloss_pct'] = df['winloss_pct'].apply(_parse_percent)

    df['price_per_saleable_sf'] = df['price_per_saleable_sf'].fillna(
        df['price_hkd'] / df['saleable_area']
    )

    df = df.dropna(subset=['date', 'price_hkd', 'saleable_area', 'price_per_saleable_sf', 'district', 'estate'])
    df = df[(df['saleable_area'] > 0) & (df['price_hkd'] > 0) & (df['price_per_saleable_sf'] > 0)]

    df['estate'] = df['estate'].astype(str).str.strip()
    df['district'] = df['district'].astype(str).str.strip()
    df['building'] = df['building'].astype(str).str.strip()

    if 'id' in df.columns:
        df = df.drop_duplicates(subset=['id'])

    return df


cleaned_df = clean_transactions(raw_df)
display(cleaned_df.head())
print(f"Cleaned rows: {len(cleaned_df):,}")
cleaned_df.to_csv(OUTPUT_DIR / 'cleaned_transactions.csv', index=False)

## 4) Macro data (World Bank)
Pull Hong Kong unemployment rate (`SL.UEM.TOTL.ZS`) and CPI (`FP.CPI.TOTL`, 2010=100) and merge by year.


In [ ]:
def fetch_world_bank_indicator(indicator_id: str, value_name: str, start_year: int = 2014, end_year: int = 2020) -> pd.DataFrame:
    url = f"https://api.worldbank.org/v2/country/HKG/indicator/{indicator_id}?per_page=500&format=json"
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    data = resp.json()
    if len(data) < 2 or not isinstance(data[1], list):
        raise ValueError(f"Unexpected World Bank response for {indicator_id}")
    records: List[Dict[str, float]] = []
    for entry in data[1]:
        year_txt = entry.get('date')
        value = entry.get('value')
        if value is None or year_txt is None:
            continue
        try:
            year = int(year_txt)
        except ValueError:
            continue
        if start_year <= year <= end_year:
            records.append({'year': year, value_name: float(value)})
    out = pd.DataFrame(records).sort_values('year').reset_index(drop=True)
    return out


def build_macro_dataset(start_year: int = 2014, end_year: int = 2020) -> pd.DataFrame:
    unemployment = fetch_world_bank_indicator('SL.UEM.TOTL.ZS', 'unemployment_rate', start_year, end_year)
    cpi = fetch_world_bank_indicator('FP.CPI.TOTL', 'cpi_index', start_year, end_year)
    return pd.merge(unemployment, cpi, on='year', how='outer')


macro_df = build_macro_dataset(2014, 2020)
display(macro_df)
macro_df.to_csv(OUTPUT_DIR / 'macro_data.csv', index=False)

## 5) Merge transactions + macro
Merge on year to enrich each transaction with unemployment/CPI.


In [ ]:
merged_df = pd.merge(cleaned_df, macro_df, on='year', how='left')
display(merged_df[['date', 'district', 'price_hkd', 'saleable_area', 'price_per_saleable_sf', 'unemployment_rate', 'cpi_index']].head())
merged_df.to_csv(OUTPUT_DIR / 'transactions_with_macro.csv', index=False)

## 6) Exploratory Data Analysis
- Summary stats & missing values
- Top districts by count
- Distribution and scatter plots
- Monthly median price trend and yearly macro overlay


In [ ]:
numeric_cols = ['price_hkd', 'saleable_area', 'gross_area', 'price_per_saleable_sf', 'price_per_gross_sf']
summary_stats = merged_df[numeric_cols].describe()
missing = merged_df[numeric_cols].isna().sum().sort_values(ascending=False)
top_districts = merged_df['district'].value_counts().head(10)

display(summary_stats)
display(missing)
display(top_districts)

plt.figure(figsize=(8, 5))
sns.histplot(x=merged_df['price_per_saleable_sf'], bins=50, kde=True, color='steelblue')
plt.xlabel('Price per saleable sqft (HKD)')
plt.title('Distribution of price per saleable sqft')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
sns.scatterplot(
    data=merged_df.sample(min(len(merged_df), 5000), random_state=42),
    x='saleable_area', y='price_per_saleable_sf', hue='district', alpha=0.6, legend=False
)
plt.title('Saleable area vs price per sqft')
plt.tight_layout()
plt.show()

monthly = merged_df.set_index('date')['price_per_saleable_sf'].resample('ME').median().dropna()
monthly.plot(figsize=(9, 4), title='Monthly median price per saleable sqft')
plt.ylabel('HKD per sqft')
plt.tight_layout()
plt.show()

yearly = (
    merged_df.groupby('year')
    .agg(median_price_per_saleable_sf=('price_per_saleable_sf', 'median'), unemployment_rate=('unemployment_rate', 'median'))
    .dropna()
)
if not yearly.empty:
    fig, ax1 = plt.subplots(figsize=(9, 4))
    ax1.plot(yearly.index, yearly['median_price_per_saleable_sf'], marker='o', color='tab:blue', label='Median price/sf')
    ax1.set_ylabel('Median price per saleable sqft (HKD)', color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.plot(yearly.index, yearly['unemployment_rate'], marker='s', color='tab:red', label='Unemployment rate')
    ax2.set_ylabel('Unemployment rate (%)', color='tab:red')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    plt.title('Yearly price per sqft vs unemployment')
    fig.tight_layout()
    plt.show()

## 7) Modeling price per saleable sqft
Two models: linear regression (baseline) and random forest. Features include size, floor, holding period, win/loss, macro indicators, and district (one-hot).


In [ ]:
target = 'price_per_saleable_sf'
feature_columns = [
    'saleable_area', 'gross_area', 'floor_num', 'holding_period_years', 'winloss_pct',
    'pre_sale_flag', 'unemployment_rate', 'cpi_index', 'year', 'district'
]

model_df = merged_df.dropna(subset=feature_columns + [target]).copy()
X = model_df[feature_columns]
y = model_df[target]

categorical_cols = ['district']
numeric_cols = [c for c in feature_columns if c not in categorical_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('numeric', 'passthrough', numeric_cols),
    ]
)

models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1, min_samples_leaf=2),
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results = []
feature_importances = None

for name, model in models.items():
    clf = Pipeline(steps=[('preprocess', preprocessor), ('model', model)])
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results.append({'model': name, 'mae': mae, 'r2': r2})

    if name == 'RandomForest':
        model_step = clf.named_steps['model']
        feat_names = clf.named_steps['preprocess'].get_feature_names_out()
        importances = pd.Series(model_step.feature_importances_, index=feat_names)
        feature_importances = importances.sort_values(ascending=False).head(15)

results_df = pd.DataFrame(results)
display(results_df)

if feature_importances is not None:
    display(feature_importances)

## 8) Findings & Next Steps
- Price per saleable sqft varies widely; distribution is right-skewed.
- Size (gross/saleable area) and district are dominant drivers; macro signals (CPI/unemployment) also contribute.
- Random forest outperforms linear regression (higher R?, lower MAE).
- Potential extensions: add more granular building/room features, include transaction type, and tune model hyperparameters.
